In [1]:
!nvidia-smi

Mon Nov 15 13:15:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/HW2/hw2_data.zip

串流輸出內容已截斷至最後 5000 行。
 extracting: hw2_data/face/train/31182.png  
 extracting: hw2_data/face/train/35296.png  
 extracting: hw2_data/face/train/29882.png  
  inflating: hw2_data/face/train/21095.png  
 extracting: hw2_data/face/train/23957.png  
  inflating: hw2_data/face/train/30313.png  
 extracting: hw2_data/face/train/11616.png  
 extracting: hw2_data/face/train/10447.png  
 extracting: hw2_data/face/train/34172.png  
 extracting: hw2_data/face/train/35917.png  
 extracting: hw2_data/face/train/18720.png  
 extracting: hw2_data/face/train/10484.png  
 extracting: hw2_data/face/train/25579.png  
 extracting: hw2_data/face/train/11294.png  
 extracting: hw2_data/face/train/21343.png  
 extracting: hw2_data/face/train/30592.png  
 extracting: hw2_data/face/train/05946.png  
 extracting: hw2_data/face/train/36061.png  
 extracting: hw2_data/face/train/07383.png  
 extracting: hw2_data/face/train/05793.png  
 extracting: hw2_data/face/train/04351.png  
 extracting: hw2_data/face/train/2

In [4]:
import os
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from PIL import Image, ImageEnhance, ImageOps
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.utils import save_image


# Dataloader

In [12]:
class Image_Dataset(Dataset):
    def __init__(self, root, train=True, trans=None):
      self.train = train
      self.root = root
      self.transform = trans
      self.data_name = pd.read_csv(self.root + self.train + '.csv')['image_name'].tolist()
      self.data_label = pd.read_csv(self.root + self.train + '.csv')['label'].tolist()

    def __getitem__(self, index):
      self.img = Image.open(self.root + self.train + '/' + self.data_name[index])
      self.img = self.transform(self.img)
      self.label = self.data_label[index]
      return (self.img, self.label)

    def __len__(self):
      return len(self.data_name)

# Architecture

In [43]:
from torch.autograd import Function

class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None

class SVHNmodel(nn.Module):
    """ SVHN architecture
    """

    def __init__(self):
        super(SVHNmodel, self).__init__()
        self.restored = False

        self.feature = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(5, 5)),  # 28
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2)),  # 13
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5, 5)),  # 9
            nn.BatchNorm2d(64),
            nn.Dropout2d(),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2)),  # 4
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(4, 4)),  # 1
        )

        self.classifier = nn.Sequential(
            nn.Linear(128 * 1 * 1, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 10),
        )

        self.discriminator = nn.Sequential(
            nn.Linear(128 * 1 * 1, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 2),
        )

    def forward(self, d_classifier, input_data, alpha = 1.0):
        input_data = input_data.expand(input_data.data.shape[0], input_data.data.shape[1], 32, 32)
        feature = self.feature(input_data)
        feature = feature.view(-1, 128 * 1 * 1)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.classifier(feature)
        if d_classifier:
          domain_output = self.discriminator(reverse_feature)

          return class_output, domain_output
        else:
          return class_output

class MNISTMmodel(nn.Module):
    """ MNIST-M architecture
    """
    def __init__(self):
        super(MNISTMmodel, self).__init__()
        self.restored = False
        self.feature = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32,
                      kernel_size=(5, 5)),  # 3 28 28, 32 24 24
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2, 2)),  # 32 12 12
            nn.Conv2d(in_channels=32, out_channels=48,
                      kernel_size=(5, 5)),  # 48 8 8
            nn.BatchNorm2d(48),
            nn.Dropout2d(),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2, 2)),  # 48 4 4
        )

        self.classifier = nn.Sequential(
            nn.Linear(48*4*4, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(inplace=True),
            nn.Linear(100, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(inplace=True),
            nn.Linear(100, 10),
        )

        self.discriminator = nn.Sequential(
            nn.Linear(48*4*4, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(inplace=True),
            nn.Linear(100, 2),
        )

    def forward(self, d_classifier, input_data, alpha = 1.0):
        input_data = input_data.expand(input_data.data.shape[0], input_data.data.shape[1], 28, 28)
        feature = self.feature(input_data)
        feature = feature.view(-1, 48 * 4 * 4)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.classifier(feature)
        if d_classifier:
          domain_output = self.discriminator(reverse_feature)

          return class_output, domain_output
        else:
          return class_output

class USPSMmodel(nn.Module):
    """ USPS architecture
    """
    def __init__(self):
        super(USPSMmodel, self).__init__()
        self.restored = False
        self.feature = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32,
                      kernel_size=(5, 5)),  # 1 28 28, 32 24 24
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2, 2)),  # 32 12 12
            nn.Conv2d(in_channels=32, out_channels=48,
                      kernel_size=(5, 5)),  # 48 8 8
            nn.BatchNorm2d(48),
            nn.Dropout2d(),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(2, 2)),  # 48 4 4
        )

        self.classifier = nn.Sequential(
            nn.Linear(48*4*4, 50),
            nn.BatchNorm1d(50),
            nn.ReLU(inplace=True),
            nn.Linear(50, 30),
            nn.BatchNorm1d(30),
            nn.ReLU(inplace=True),
            nn.Linear(30, 10),
        )

        self.discriminator = nn.Sequential(
            nn.Linear(48*4*4, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(inplace=True),
            nn.Linear(100, 2),
        )

    def forward(self, d_classifier, input_data, alpha = 1.0):
        input_data = input_data.expand(input_data.data.shape[0], input_data.data.shape[1], 28, 28)
        feature = self.feature(input_data)
        feature = feature.view(-1, 48 * 4 * 4)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.classifier(feature)
        if d_classifier:
          domain_output = self.discriminator(reverse_feature)

          return class_output, domain_output
        else:
          return class_output

        

In [6]:

# Init weights for layers w.r.t. the original paper.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# Training Function

In [8]:
def adjust_learning_rate(optimizer, p):
    lr_0 = 0.01
    alpha = 10
    beta = 0.75
    lr = lr_0 / (1 + alpha * p)**beta
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

def save_model(net, filename):
    """Save trained model."""
    torch.save(net.state_dict(), '/content/drive/MyDrive/HW2/model/DANN/%s'%filename)
    print("save pretrained model successful")

def train_on_source(model, src_data_loader, tgt_data_loader, tgt_data_loader_eval, device, n_epochs, l_step, save_modle_name):

    ####################
    # 1. train network #
    ####################
    global_step = 0
    for epoch in range(n_epochs):
        # set train state for Dropout and BN layers
        model.train()
        # zip source and target data pair
        len_dataloader = len(src_data_loader)

        for step, ((images_src, class_src)) in enumerate(src_data_loader):

            p = float(step + epoch * len_dataloader) / n_epochs / len_dataloader
            alpha = 2. / (1. + np.exp(-10 * p)) - 1
            lr = adjust_learning_rate(optimizer, p)

            # make images variable
            class_src = class_src.to(device)
            images_src = images_src.to(device)

            # zero gradients for optimizer
            model.zero_grad()

            # train on source domain
            src_class_output = model(d_classifier=False, input_data=images_src, alpha=alpha)
            src_loss_class = criterion(src_class_output, class_src)

            loss = src_loss_class

            # optimize dann
            loss.backward()
            optimizer.step()

            global_step += 1

            # print step info

            if ((step + 1) % l_step == 0):
                print(
                    "Epoch [{:4d}/{}] Step [{:2d}/{}]: loss={:.6f}".format(epoch + 1, n_epochs, step + 1, len_dataloader, loss.data.item()))

        # eval model
        if ((epoch + 1) % 1 == 0):
            src_test_loss, src_acc = test(model, src_data_loader, device, flag='source', train_source=True)
            tgt_test_loss, tgt_acc = test(model, tgt_data_loader_eval, device, flag='target', train_source=True)

    #     # save model parameters
    #     if ((epoch + 1) % params.save_step == 0):
    #         save_model(model, params.model_root,
    #                    params.src_dataset + '-' + params.tgt_dataset + "-dann-{}.pt".format(epoch + 1))

    # save final model
    save_model(model, save_modle_name)

    return model

def test(model, data_loader, device, flag, train_source):
    """Evaluate model for dataset."""
    # set eval state for Dropout and BN layers
    model.eval()

    # init loss and accuracy
    loss_ = 0.0
    acc_ = 0.0
    acc_domain_ = 0.0
    n_total = 0

    # evaluate network
    for (images, labels) in data_loader:
        images = images.to(device)
        labels = labels.to(device)  #labels = labels.squeeze(1)
        size = len(labels)
        if flag == 'target':
            labels_domain = torch.ones(size).long().to(device)
            
        else:
            labels_domain = torch.zeros(size).long().to(device)

        if train_source:
          preds = model(d_classifier=False, input_data=images, alpha=0)
        
        else:
          preds, domain = model(d_classifier=True, input_data=images, alpha=0)
          pred_domain = domain.data.max(1)[1]
          acc_domain_ += pred_domain.eq(labels_domain.data).sum().item()
          acc_domain = acc_domain_ / n_total
        
        loss_ += criterion(preds, labels).item()
        pred_cls = preds.data.max(1)[1]
        acc_ += pred_cls.eq(labels.data).sum().item()
        
        n_total += size

    loss = loss_ / n_total
    acc = acc_ / n_total
    
    try:
      print("Avg Loss = {:.6f}, Avg Accuracy = {:.2%}, {}/{}, Avg Domain Accuracy = {:2%}".format(loss, acc, acc_, n_total, acc_domain))
    except:
      print("Avg Loss = {:.6f}, Avg Accuracy = {:.2%}, {}/{}".format(loss, acc, acc_, n_total))

    if train_source:
      return loss, acc
    else:
      return loss, acc, acc_domain

# SVHN → MNIST-M

In [46]:
image_size = 32
batch_size = 64
num_epochs = 10
lr = 0.01
momentum = 0.9
weight_decay = 1e-6
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")


In [47]:
tfm_3 = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

tfm_1 = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3,1,1)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

## Prepare dataset

In [48]:
s_train_set = Image_Dataset('/content/hw2_data/digits/svhn/', train='train', trans=tfm_3)
s_test_set = Image_Dataset('/content/hw2_data/digits/svhn/', train='test', trans=tfm_3)

t_train_set = Image_Dataset('/content/hw2_data/digits/mnistm/', train='train', trans=tfm_3)
t_test_set = Image_Dataset('/content/hw2_data/digits/mnistm/', train='test', trans=tfm_3)

s_data_loader = DataLoader(s_train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
s_data_loader_eval = DataLoader(s_test_set, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)
t_data_loader = DataLoader(t_train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
t_data_loader_eval = DataLoader(t_test_set, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

dann = SVHNmodel()
dann.apply(weights_init)
dann.to(device)

optimizer = optim.SGD(dann.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## Training on source domain only

In [14]:
dann = train_on_source(dann, s_data_loader, t_data_loader, t_data_loader_eval, device, num_epochs, 200, 'source_only_SVHN_to_MNIST-M.pth')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch [   1/10] Step [200/1145]: loss=1.040673
Epoch [   1/10] Step [400/1145]: loss=1.137965
Epoch [   1/10] Step [600/1145]: loss=0.553148
Epoch [   1/10] Step [800/1145]: loss=0.818993
Epoch [   1/10] Step [1000/1145]: loss=0.785944
Avg Loss = 0.007030, Avg Accuracy = 86.20%, 63148.0/73257
Avg Loss = 0.024955, Avg Accuracy = 47.49%, 4749.0/10000
Epoch [   2/10] Step [200/1145]: loss=0.651600
Epoch [   2/10] Step [400/1145]: loss=0.601969
Epoch [   2/10] Step [600/1145]: loss=0.502170
Epoch [   2/10] Step [800/1145]: loss=0.578168
Epoch [   2/10] Step [1000/1145]: loss=0.542281
Avg Loss = 0.005528, Avg Accuracy = 89.32%, 65435.0/73257
Avg Loss = 0.025501, Avg Accuracy = 48.00%, 4800.0/10000
Epoch [   3/10] Step [200/1145]: loss=0.323853
Epoch [   3/10] Step [400/1145]: loss=0.443671
Epoch [   3/10] Step [600/1145]: loss=0.486709
Epoch [   3/10] Step [800/1145]: loss=0.525514
Epoch [   3/10] Step [1000/1145]: loss=0.347419
Avg Loss = 0.004868, Avg Accuracy = 90.66%, 66418.0/73257
Avg 

## Predict on source domain


In [49]:
dann = SVHNmodel()
dann.load_state_dict(torch.load('/content/drive/MyDrive/HW2/model/DANN/source_only_SVHN_to_MNIST-M.pth'))

test(dann.to(device), s_data_loader_eval, device, flag='target', train_source=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Avg Loss = 0.004528, Avg Accuracy = 91.55%, 23832.0/26032


(0.0045283855491139785, 0.9154886293792256)

# Predict on traget domain

In [32]:
dann = SVHNmodel()
dann.load_state_dict(torch.load('/content/drive/MyDrive/HW2/model/DANN/source_only_SVHN_to_MNIST-M.pth'))

test(dann.to(device), t_data_loader_eval, device, flag='target', train_source=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Avg Loss = 0.027092, Avg Accuracy = 49.25%, 4925.0/10000


(0.027092498284578322, 0.4925)

# MNIST-M → USPS

In [50]:
image_size = 28
batch_size = 64
num_epochs = 10
lr = 0.01
momentum = 0.9
weight_decay = 1e-6
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")


In [51]:
tfm_3 = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

tfm_1 = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3,1,1)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

## Prepare dataset

In [52]:
s_train_set = Image_Dataset('/content/hw2_data/digits/mnistm/', train='train', trans=tfm_3)
s_test_set = Image_Dataset('/content/hw2_data/digits/mnistm/', train='test', trans=tfm_3)

t_train_set = Image_Dataset('/content/hw2_data/digits/usps/', train='train', trans=tfm_1)
t_test_set = Image_Dataset('/content/hw2_data/digits/usps/', train='test', trans=tfm_1)

s_data_loader = DataLoader(s_train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
s_data_loader_eval = DataLoader(s_test_set, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)
t_data_loader = DataLoader(t_train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
t_data_loader_eval = DataLoader(t_test_set, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

# dann = init_model(net=SVHNmodel(d_classifier=False), restore=None)

dann = MNISTMmodel()
dann.apply(weights_init)
dann.to(device)

optimizer = optim.SGD(dann.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## Training on source domain only

In [21]:
dann = train_on_source(dann, s_data_loader, t_data_loader, t_data_loader_eval, device, num_epochs, 200, 'source_only_MNIST-M_to_USPS.pth')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch [   1/10] Step [200/938]: loss=0.555174
Epoch [   1/10] Step [400/938]: loss=0.323610
Epoch [   1/10] Step [600/938]: loss=0.258256
Epoch [   1/10] Step [800/938]: loss=0.196938
Avg Loss = 0.002363, Avg Accuracy = 95.52%, 57309.0/60000
Avg Loss = 0.015338, Avg Accuracy = 73.64%, 1478.0/2007
Epoch [   2/10] Step [200/938]: loss=0.262066
Epoch [   2/10] Step [400/938]: loss=0.331580
Epoch [   2/10] Step [600/938]: loss=0.113572
Epoch [   2/10] Step [800/938]: loss=0.112764
Avg Loss = 0.001774, Avg Accuracy = 96.47%, 57882.0/60000
Avg Loss = 0.014665, Avg Accuracy = 75.54%, 1516.0/2007
Epoch [   3/10] Step [200/938]: loss=0.232701
Epoch [   3/10] Step [400/938]: loss=0.161456
Epoch [   3/10] Step [600/938]: loss=0.102827
Epoch [   3/10] Step [800/938]: loss=0.202160
Avg Loss = 0.001487, Avg Accuracy = 97.08%, 58250.0/60000
Avg Loss = 0.016902, Avg Accuracy = 73.09%, 1467.0/2007
Epoch [   4/10] Step [200/938]: loss=0.095094
Epoch [   4/10] Step [400/938]: loss=0.099395
Epoch [   4/10

# Predict on source domain

In [53]:
dann = MNISTMmodel()
dann.load_state_dict(torch.load('/content/drive/MyDrive/HW2/model/DANN/source_only_MNIST-M_to_USPS.pth'))

test(dann.to(device), s_data_loader_eval, device, flag='target', train_source=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Avg Loss = 0.001048, Avg Accuracy = 97.98%, 9798.0/10000


(0.0010484198365942575, 0.9798)

## Predict on traget domain

In [36]:
dann = MNISTMmodel()
dann.load_state_dict(torch.load('/content/drive/MyDrive/HW2/model/DANN/source_only_MNIST-M_to_USPS.pth'))

test(dann.to(device), t_data_loader_eval, device, flag='target', train_source=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Avg Loss = 0.016868, Avg Accuracy = 74.29%, 1491.0/2007


(0.01686762557673585, 0.7428998505231689)

# USPS → SVHN

In [56]:
image_size = 28
batch_size = 64
num_epochs = 10
lr = 0.01
momentum = 0.9
weight_decay = 1e-6
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")


In [57]:
tfm_3 = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

tfm_1 = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3,1,1)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

In [58]:
s_train_set = Image_Dataset('/content/hw2_data/digits/usps/', train='train', trans=tfm_1)
s_test_set = Image_Dataset('/content/hw2_data/digits/usps/', train='test', trans=tfm_1)

t_train_set = Image_Dataset('/content/hw2_data/digits/svhn/', train='train', trans=tfm_3)
t_test_set = Image_Dataset('/content/hw2_data/digits/svhn/', train='test', trans=tfm_3)

s_data_loader = DataLoader(s_train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
s_data_loader_eval = DataLoader(s_test_set, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)
t_data_loader = DataLoader(t_train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
t_data_loader_eval = DataLoader(t_test_set, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

# dann = init_model(net=SVHNmodel(d_classifier=False), restore=None)

dann = USPSMmodel()
dann.apply(weights_init)
dann.to(device)

optimizer = optim.SGD(dann.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## Training on source domain only

In [45]:
dann = train_on_source(dann, s_data_loader, t_data_loader, t_data_loader_eval, device, num_epochs, 200, 'source_only_USPS_to_SVHN.pth')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Avg Loss = 0.004324, Avg Accuracy = 95.17%, 6939.0/7291
Avg Loss = 0.036278, Avg Accuracy = 14.79%, 3851.0/26032
Avg Loss = 0.001812, Avg Accuracy = 97.89%, 7137.0/7291
Avg Loss = 0.036394, Avg Accuracy = 21.14%, 5504.0/26032
Avg Loss = 0.001223, Avg Accuracy = 98.51%, 7182.0/7291
Avg Loss = 0.038148, Avg Accuracy = 21.38%, 5565.0/26032
Avg Loss = 0.000988, Avg Accuracy = 98.74%, 7199.0/7291
Avg Loss = 0.038951, Avg Accuracy = 20.02%, 5211.0/26032
Avg Loss = 0.000755, Avg Accuracy = 99.07%, 7223.0/7291
Avg Loss = 0.041267, Avg Accuracy = 18.05%, 4699.0/26032
Avg Loss = 0.000641, Avg Accuracy = 99.19%, 7232.0/7291
Avg Loss = 0.043989, Avg Accuracy = 17.69%, 4604.0/26032
Avg Loss = 0.000557, Avg Accuracy = 99.29%, 7239.0/7291
Avg Loss = 0.044635, Avg Accuracy = 16.43%, 4277.0/26032
Avg Loss = 0.000500, Avg Accuracy = 99.38%, 7246.0/7291
Avg Loss = 0.047720, Avg Accuracy = 14.99%, 3901.0/26032
Avg Loss = 0.000445, Avg Accuracy = 99.53%, 7257.0/7291
Avg Loss = 0.046302, Avg Accuracy = 14.5

# Predict on source domain

In [60]:
dann = USPSMmodel()
dann.load_state_dict(torch.load('/content/drive/MyDrive/HW2/model/DANN/source_only_USPS_to_SVHN.pth'))

test(dann.to(device), s_data_loader_eval, device, flag='target', train_source=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Avg Loss = 0.002108, Avg Accuracy = 96.26%, 1932.0/2007


(0.0021082623808365824, 0.9626307922272048)

## Predict on traget domain

In [59]:
dann = USPSMmodel()
dann.load_state_dict(torch.load('/content/drive/MyDrive/HW2/model/DANN/source_only_USPS_to_SVHN.pth'))

test(dann.to(device), t_data_loader_eval, device, flag='target', train_source=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Avg Loss = 0.046437, Avg Accuracy = 15.26%, 3973.0/26032


(0.04643707476515087, 0.152619852489244)